##**Лабораторна робота №3**
##Тема. Побудова композитних індикаторів (Частина 1)

**Завантаження пакетів та визначення індикаторів**

In [4]:
install.packages("WDI")
library(WDI)
library(dplyr)

# Визначаємо індикатори Світового банку, які будемо завантажувати.
# Кожен показник відповідає складовій економічної безпеки.
indicators <- c(
  gdp_pc      = "NY.GDP.PCAP.KD",        # ВВП на душу населення (сталий $, показник рівня розвитку)
  unemployment= "SL.UEM.TOTL.ZS",        # Рівень безробіття (% робочої сили)
  inflation   = "FP.CPI.TOTL.ZG",        # Інфляція (річна %)
  trade       = "NE.TRD.GNFS.ZS",        # Торгівля як % ВВП (відкритість економіки)
  fdi         = "BX.KLT.DINV.WD.GD.ZS",  # Прямі іноземні інвестиції (% ВВП)
  gov_debt    = "GC.DOD.TOTL.GD.ZS"      # Державний борг (% ВВП)
)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



**Завантаження даних з WDI та збереження сирого датасету**

In [5]:
# Завантаження сирих даних з WDI за всіма доступними країнами за 2005–2020 роки.
# extra = TRUE -> додає додаткову інформацію про країну (регіон, доходи тощо)
raw_data <- WDI(
  country = "all",
  indicator = indicators,
  start = 2005,
  end = 2020,
  extra = TRUE
)

# Видаляємо агреговані регіони (наприклад "Східна Європа", "Єврозона"),
# бо для аналізу потрібні лише окремі країни.
raw_data <- raw_data %>% filter(region != "Aggregates")

# Зберігаємо початковий датасет у CSV для повторного використання.
write.csv(raw_data, "raw_economic_security_data.csv", row.names = FALSE)


**Завантаження сирих даних і вибір потрібних колонок**

In [6]:
install.packages("zoo")
library(dplyr)
library(tidyr)
library(zoo)

# Завантаження сирих даних із файлу
raw <- read.csv("raw_economic_security_data.csv")

# Вибираємо лише потрібні колонки, включаючи категоріальні змінні (income, lending),
# та обмежуємо роки до заданого періоду.
data <- raw %>%
  select(country, iso3c, year,
         unemployment, trade, fdi, gov_debt,
         income, lending) %>%                     # income/lending — групи країн за класифікацією Світового банку
  filter(year >= 2005 & year <= 2020)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




**Перетворення категоріальних змінних у числові**

In [7]:
# Це важливо для PCA та кореляційного аналізу, які працюють лише з числами.
income_map <- c(
  "Low income" = 1,
  "Lower middle income" = 2,
  "Upper middle income" = 3,
  "High income" = 4
)

lending_map <- c(
  "IDA" = 1,    # Фонд найнижчих доходів
  "Blend" = 2,  # Перехідні країни
  "IBRD" = 3    # Кредитування для країн з вищим доходом
)

# Додаємо числові колонки за категоріальними значеннями.
data <- data %>%
  mutate(
    income_num  = income_map[income],      # Перетворення дохідної категорії у число
    lending_num = lending_map[lending]     # Перетворення кредитного статусу у число
  )


**Заповнення пропусків у даних**

In [8]:
data_filled <- data %>%
  arrange(iso3c, year) %>%                 # Впорядкування для коректної інтерполяції
  group_by(iso3c) %>%                      # Заповнюємо пропуски окремо для кожної країни
  mutate(
    across(
      c(unemployment, trade, fdi, gov_debt, income_num, lending_num),
      ~ na.approx(., na.rm = FALSE)        # Лінійна інтерполяція значень у часі
    )
  ) %>%
  ungroup() %>%
  mutate(
    across(
      c(unemployment, trade, fdi, gov_debt, income_num, lending_num),
      ~ ifelse(is.na(.), median(., na.rm = TRUE), .)  # Якщо залишились NA → замінюємо медіаною
    )
  )


**Кореляційний аналіз**

In [9]:
# Дозволяє визначити взаємозв'язки між індикаторами
cor_matrix <- data_filled %>%
  select(unemployment, trade, fdi, gov_debt, income_num, lending_num) %>%
  cor()

cor_matrix  # Вивід кореляційної матриці


,unemployment,trade,fdi,gov_debt,income_num,lending_num
unemployment,1.000000000,-0.002995697,-0.015756409,-0.007725699,0.03029350,0.223895272
trade,-0.002995697,1.000000000,0.054728345,0.067279840,0.31408059,0.060712309
fdi,-0.015756409,0.054728345,1.000000000,0.003647828,0.08815625,-0.002828111
gov_debt,-0.007725699,0.067279840,0.003647828,1.000000000,0.10108764,-0.001128524
income_num,0.030293496,0.314080592,0.088156251,0.101087639,1.00000000,0.476325086
lending_num,0.223895272,0.060712309,-0.002828111,-0.001128524,0.47632509,1.000000000


**PCA (Головні компоненти)**

In [10]:
# Метод редукції вимірності, який допомагає зрозуміти структуру даних та домінантні фактори
pca_result <- prcomp(
  data_filled %>%
    select(unemployment, trade, fdi, gov_debt, income_num, lending_num),
  scale. = TRUE                                # Масштабування важливе для коректної PCA
)

summary(pca_result)       # Пояснена дисперсія
pca_result$rotation       # Внесок кожної змінної у компоненти


Importance of components:
                          PC1    PC2    PC3    PC4    PC5    PC6
Standard deviation     1.2878 1.0677 0.9983 0.9687 0.9172 0.6522
Proportion of Variance 0.2764 0.1900 0.1661 0.1564 0.1402 0.0709
Cumulative Proportion  0.2764 0.4664 0.6325 0.7889 0.9291 1.0000

,PC1,PC2,PC3,PC4,PC5,PC6
unemployment,0.2225348,0.6238363,0.01534465,0.43840183,0.57333304,0.20040976
trade,0.3881136,-0.4576737,0.02534768,-0.30142768,0.67981487,-0.29368897
fdi,0.1127364,-0.3481359,-0.73247387,0.56674365,-0.05251549,-0.07495140
gov_debt,0.1370640,-0.3606942,0.67951945,0.60760074,-0.11377686,-0.08510354
income_num,0.6538666,-0.1306278,-0.01299050,-0.15733166,-0.23230127,0.69029630
lending_num,0.5837881,0.3646923,-0.02624183,-0.04260176,-0.37346223,-0.61984950


**Збереження оброблених даних**

In [11]:
write.csv(data_filled, "processed_economic_security_data.csv", row.names = FALSE)

**Нормалізація індикаторів**

In [12]:
library(dplyr)
library(zoo)

# Завантажуємо фінальні очищені дані
data_filled <- read.csv("processed_economic_security_data.csv")

# Нормалізація індикаторів
# Використовується Z-score: (x - mean) / sd
normalize <- function(x) {
  return((x - mean(x)) / sd(x))
}

data_norm <- data_filled %>%
  mutate(
    # Дестимулятори: чим менше — тим краще, тому інвертуємо знак
    unemployment_z = -normalize(unemployment),  # Високе безробіття = гірше
    gov_debt_z     = -normalize(gov_debt),      # Високий борг = ризик

    # Стимулятори: більше → краще
    trade_z        = normalize(trade),          # Відкритість економіки
    fdi_z          = normalize(fdi),            # Інвестиції
    income_z       = normalize(income_num),     # Рівень доходів
    lending_z      = normalize(lending_num)     # Тип кредитування
  )


**Побудова композитного індексу**

In [13]:
# Просте середнє нормалізованих значень — базова версія інтегрального індексу
data_index <- data_norm %>%
  mutate(
    economic_security_index = (
      unemployment_z +
      trade_z +
      fdi_z +
      gov_debt_z +
      income_z +
      lending_z
    ) / 6
  )


**Додавання рангів**

In [14]:
# Чим більший індекс — тим вищий ранг безпеки
data_index <- data_index %>%
  group_by(year) %>%
  mutate(rank = rank(-economic_security_index, ties.method = "min")) %>%
  ungroup()


**Збереження фінальних результатів та перегляд**

In [15]:
# Збереження фінального результату
write.csv(data_index, "final_economic_security_index.csv", row.names = FALSE)

# Перегляд перших рядків індексу
head(data_index)


country,iso3c,year,unemployment,trade,fdi,gov_debt,income,lending,income_num,lending_num,unemployment_z,gov_debt_z,trade_z,fdi_z,income_z,lending_z,economic_security_index,rank
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Aruba,ABW,2005,6.0875,145.0503,-8.8049356,46.73797,High income,Not classified,4,2,0.2842933,0.07856971,1.0079810,-0.27351207,1.034945,-0.04865595,0.3472702,32
Aruba,ABW,2006,6.0875,141.0426,8.9204666,46.73797,High income,Not classified,4,2,0.2842933,0.07856971,0.9354738,-0.01526802,1.034945,-0.04865595,0.3782263,32
Aruba,ABW,2007,6.0875,139.9723,-17.5903165,46.73797,High income,Not classified,4,2,0.2842933,0.07856971,0.9161094,-0.40150760,1.034945,-0.04865595,0.3106257,52
Aruba,ABW,2008,6.0875,139.1101,0.6641758,46.73797,High income,Not classified,4,2,0.2842933,0.07856971,0.9005110,-0.13555516,1.034945,-0.04865595,0.3523513,46
Aruba,ABW,2009,6.0875,137.5566,-0.4156376,46.73797,High income,Not classified,4,2,0.2842933,0.07856971,0.8724034,-0.15128712,1.034945,-0.04865595,0.3450447,37
Aruba,ABW,2010,6.0875,135.4647,7.6116722,46.73797,High income,Not classified,4,2,0.2842933,0.07856971,0.8345560,-0.03433604,1.034945,-0.04865595,0.3582287,37


##**Лабораторна робота №4**
##Тема. Побудова композитних індикаторів (Частина 2)

**Підготовка даних та визначення стимуляторів / дестимуляторів**

In [16]:
econ <- data_index   # робоча копія таблиці


# Визначаємо індикатори-стимулятори та дестимулятори

# Стимулятори — це змінні, де більші значення підвищують рівень економічної безпеки
# (наприклад, торгівля, FDI, доходи). Дестимулятори — навпаки, погіршують стан (безробіття, борг).

# z-оцінки, де "більше = краще" (стимулятори)
stim_z <- c(
  "trade_z",    # інтенсивність торгівлі
  "fdi_z",      # іноземні інвестиції
  "income_z",   # рівень доходу
  "lending_z"   # умови кредитування
)

# z-оцінки, де "більше = гірше" (дестимулятори)
destim_z <- c(
  "unemployment_z",  # безробіття — високі значення означають погіршення
  "gov_debt_z"       # державний борг — високі значення підвищують ризик
)


**Інверсія дестимуляторів та формування повного набору індикаторів**

In [17]:
# Перетворюємо дестимулятори в "позитивні" показники
# (щоб потім усі індикатори трактувалися однаково: "більше = краще")
# Додаємо суфікс *_rev, що означає інверсію початкових значень

for (v in destim_z) {
  econ[[paste0(v, "_rev")]] <- -econ[[v]]   # інверсія знака
}

# Формуємо список усіх індикаторів після інверсії
all_indicators <- c(
  stim_z,
  paste0(destim_z, "_rev")   # замість дестимулятора використовуємо його інверсію
)


**Задання ваг та обчислення інтегрального індексу**

In [18]:
# Задаємо ваги (рівні для всіх індикаторів)
# Простий рівноважний варіант: кожен індикатор має однакову важливість у моделі

n_ind <- length(all_indicators)          # кількість індикаторів
weights <- rep(1 / n_ind, n_ind)         # рівні ваги
names(weights) <- all_indicators         # назви ваг відповідають назві індикатора


# Розрахунок інтегрального індексу
# Створюємо колонку з початковим нульовим значенням індексу
econ$econ_security_index_step1 <- 0

# Додаємо внесок кожного індикатора за формулою: Σ (вага * нормалізоване значення)
for (v in all_indicators) {
  econ$econ_security_index_step1 <-
    econ$econ_security_index_step1 + weights[v] * econ[[v]]
}


**Формування рейтингу країн та перевірка кореляції з попереднім індексом**

In [19]:
# Формуємо рейтинг країн
# Вибираємо основні поля та сортуємо за спаданням індексу (вищий індекс = краща безпека)

final_index <- econ[, c("country", "year", "econ_security_index_step1")]
final_index <- final_index[order(-final_index$econ_security_index_step1), ]


head(final_index, 10)   # перегляд топ-10 країн

# Порівняння нового індексу з попереднім індексом
cor(econ$economic_security_index, econ$econ_security_index_step1, #наскільки новий індекс схожий на вже наявний
     use = "complete.obs")


country,year,econ_security_index_step1
<chr>,<int>,<dbl>
Cayman Islands,2015,4.199384
Liechtenstein,2015,3.162012
Liechtenstein,2006,2.754865
Virgin Islands (U.S.),2008,2.544620
Cayman Islands,2014,2.497717
Liechtenstein,2016,2.409482
Seychelles,2008,2.051972
Virgin Islands (U.S.),2011,2.044638
Cayman Islands,2013,1.971500


[1] 0.5044325

**Підготовка до аналізу чутливості моделі**

In [20]:
# Перевірка на чутливість та адекватність моделі

econ <- data_index   # повторно створюємо робочу копію

# Індикатори, які використовуємо (z-оцінки)
stim_z   <- c("trade_z", "fdi_z", "income_z", "lending_z")
destim_z <- c("unemployment_z", "gov_debt_z")

# Підстрахуємось: створимо *_rev, якщо їх ще нема
# (це дозволяє виконувати код у будь-якому порядку без помилок)
for (v in destim_z) {
  rev_name <- paste0(v, "_rev")
  if (!rev_name %in% names(econ)) {
    econ[[rev_name]] <- -econ[[v]]
  }
}

# Повний список індикаторів після інверсії дестимуляторів
all_ind <- c(stim_z, paste0(destim_z, "_rev"))
n_ind   <- length(all_ind)


**Розрахунок базового індексу при необхідності**

In [21]:
# Якщо індекс ще не обчислений — обчислюємо базовий
# Цей блок потрібен для повторних запусків або тестів чутливості

if (!"econ_security_index_step1" %in% names(econ)) {
  base_w <- rep(1 / n_ind, n_ind)
  names(base_w) <- all_ind

  econ$econ_security_index_step1 <- 0
  for (v in names(base_w)) {
    econ$econ_security_index_step1 <-
      econ$econ_security_index_step1 + base_w[v] * econ[[v]]
  }
}


**Генерація наборів ваг для аналізу чутливості**

In [22]:
set.seed(1)  # фіксуємо генератор випадкових чисел для відтворюваності результатів

base_w <- rep(1 / n_ind, n_ind)
names(base_w) <- all_ind

# Створюємо 100 наборів ваг, випадково варіюючи кожну вагу в межах 0.8–1.2
weights_sensitivity <- lapply(1:100, function(i) {
  w <- base_w * runif(n_ind, 0.8, 1.2)    # випадкові множники
  w / sum(w)                              # нормалізація ваг, щоб сума = 1
})


**Функція розрахунку індексу та обчислення кореляцій**

In [23]:
# Функція розрахунку індексу за заданим набором ваг
calc_index <- function(w) {
  idx <- rep(0, nrow(econ))
  for (v in names(w)) {
    idx <- idx + w[v] * econ[[v]]
  }
  idx
}

# Порівнюємо кожен новий індекс із базовим через кореляцію
cor_values <- sapply(weights_sensitivity, function(w) {
  idx_new <- calc_index(w)
  cor(idx_new, econ$econ_security_index_step1, use = "complete.obs")
})

# Дивимось характеристики кореляцій — чим вищі значення, тим стабільніша модель
summary(cor_values)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.9921  0.9959  0.9968  0.9967  0.9978  0.9994 

**Аналіз важливості кожного індикатора**

In [24]:
# Вплив кожного індикатора
# Видаляємо один індикатор, переобчислюємо індекс і дивимося,
# наскільки "просів" зв'язок із базовим індексом.

impact <- list()

for (ind in all_ind) {

  subset_ind <- setdiff(all_ind, ind)   # набір індикаторів без одного
  w <- rep(1 / length(subset_ind), length(subset_ind))
  names(w) <- subset_ind

  idx_new <- rep(0, nrow(econ))
  for (v in names(w)) {
    idx_new <- idx_new + w[v] * econ[[v]]
  }

  impact[[ind]] <- cor(idx_new, econ$econ_security_index_step1,
                       use = "complete.obs")   # кореляція зі старою моделлю
}

impact   # показує важливість кожного індикатора

$trade_z
[1] 0.9434978

$fdi_z
[1] 0.9414599

$income_z
[1] 0.951982

$lending_z
[1] 0.9468451

$unemployment_z_rev
[1] 0.9417565

$gov_debt_z_rev
[1] 0.9415443

**Порівняння з попереднім індексом та збереження CSV**

In [25]:
#  Порівняння з попереднім індексом
# Якщо у таблиці є колонка economic_security_index з попередньої моделі,
# порівнюємо її з новим інтегральним індексом.

if ("economic_security_index" %in% names(econ)) {
  cor_old_new <- cor(
    econ$economic_security_index,
    econ$econ_security_index_step1,
    use = "complete.obs"
  )
  cor_old_new
} else {
  cor_old_new <- NA
  message("У таблиці немає колонки economic_security_index")
}

# Зберігаємо результати у CSV
readr::write_csv(data_index, "data_index.csv")

[1] 0.5044325